# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json
from scipy.stats import linregress

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

In [2]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

621

In [14]:
# cities

In [15]:
# Get weather data
weather_response = requests.get(query_url)
weather_json = weather_response.json()
weather_json

{'coord': {'lon': -139.0333, 'lat': -9.8},
 'weather': [{'id': 800,
   'main': 'Clear',
   'description': 'clear sky',
   'icon': '01d'}],
 'base': 'stations',
 'main': {'temp': 79.36,
  'feels_like': 77.94,
  'temp_min': 79.36,
  'temp_max': 79.36,
  'pressure': 1011,
  'humidity': 77,
  'sea_level': 1011,
  'grnd_level': 1003},
 'visibility': 10000,
 'wind': {'speed': 17.47, 'deg': 100},
 'clouds': {'all': 0},
 'dt': 1612994940,
 'sys': {'country': 'PF', 'sunrise': 1612970219, 'sunset': 1613015035},
 'timezone': -34200,
 'id': 4020109,
 'name': 'Atuona',
 'cod': 200}

In [4]:
# weather_json

In [5]:
weather_json['main']['temp']

79.36

In [6]:
temperature = weather_json["main"]["temp"]

In [7]:
print(f"The temperature in Atuona is {temperature} F.")

The temperature in Atuona is 79.36 F.


In [24]:
cities = cities[0:10]
temp = []
lat = []

url = "http://api.openweathermap.org/data/2.5/weather?&units=imperial"
# city = "atuona"

# city_url = url + '&q=' + city + '&appid=' + 'weather_api_key'
base_url = f"{url}&appid={weather_api_key}"


for city in cities:
    my_city_url = base_url + '&q=' + city
    print(my_city_url)
    response = requests.get(my_city_url).json()
#     temp.append(response['main']['temp'])
#     lat.append(response['coord']['lat'])
    time.sleep(1)
    
print(response)
# print(f"The temperature information received is: {temp}")
# print(f"The latitude information received is: {lat}")

http://api.openweathermap.org/data/2.5/weather?&units=imperial&appid=2be39acdfcd7e58b6512350cc6bafd38&q=albany
http://api.openweathermap.org/data/2.5/weather?&units=imperial&appid=2be39acdfcd7e58b6512350cc6bafd38&q=qaanaaq
http://api.openweathermap.org/data/2.5/weather?&units=imperial&appid=2be39acdfcd7e58b6512350cc6bafd38&q=bluff
http://api.openweathermap.org/data/2.5/weather?&units=imperial&appid=2be39acdfcd7e58b6512350cc6bafd38&q=dikson
http://api.openweathermap.org/data/2.5/weather?&units=imperial&appid=2be39acdfcd7e58b6512350cc6bafd38&q=barrow
http://api.openweathermap.org/data/2.5/weather?&units=imperial&appid=2be39acdfcd7e58b6512350cc6bafd38&q=mys shmidta
http://api.openweathermap.org/data/2.5/weather?&units=imperial&appid=2be39acdfcd7e58b6512350cc6bafd38&q=inuvik
http://api.openweathermap.org/data/2.5/weather?&units=imperial&appid=2be39acdfcd7e58b6512350cc6bafd38&q=krutinka
http://api.openweathermap.org/data/2.5/weather?&units=imperial&appid=2be39acdfcd7e58b6512350cc6bafd38&q=c

In [25]:
# Create a dataframe from the cities, lat, and temp
weather_dict = {
        "city": cities,
        "lat": lat,
        "temp": temp
}
weather_data = pd.DataFrame(weather_dict)
weather_data.head()

ValueError: arrays must all be same length

## Generate Cities List

In [10]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []
lat_range

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

614

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [11]:
#  Get the indices of cities that have humidity over 100%.


In [12]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression